# Steal The Files - Dokumentation (DE)
<span style="color:#4CAF50;font-weight:600">Client</span> | <span style="color:#03A9F4;font-weight:600">Server</span> | <span style="color:#FF9800;font-weight:600">Core</span> - JavaFX 2D-Plattformer

**Inhalt**
1. &Uuml;berblick &amp; Architektur
2. Setup &amp; Start
3. Screens &amp; Navigation
4. Steuerung, Kamera &amp; HUD
5. Gameplay-Objekte &amp; Mechaniken
6. Leveldesign
7. Audio &amp; Settings
8. Netzwerk &amp; Daten
9. Debugging &amp; To-dos

## 1. &Uuml;berblick &amp; Architektur
- Ziel: USB-Stick holen -> Dateien (Folder) sichern -> Gefahren &uuml;berleben -> Finish-Portal erreichen.
- Module: **core** (Entity-Basis, Events, Packets/Serialization), **client** (JavaFX-Screens, Level, Audio, Input), **server** (TCP-Lobby + Event-Router + DB-Hooks).
- Start (Game.start): Config laden, Sound-Einstellungen anwenden, EventListener registrieren, ScreenManager + MainMenu/Settings instanziieren, aktuelles Level aus `config.json` setzen.
- Game-Loop: eigener Daemon-Thread (~200 Hz) im ScreenManager berechnet `delta` + FPS und ruft `currentScreen.update(delta)` auf dem JavaFX-Thread.
- Datenfluss Spielrunde:
```text
Input (Scene events) -> InputManager -> ScreenManager.update(delta) -> GameScreen.update(delta)
  -> Physik (dx/dy, Gravity)
  -> Kollision (Platform + Block)
  -> Kamera (Dead-Zone + smoothing)
  -> HUD (Hearts, Quest, Debug)
  -> Render + Audio (SoundManager)
```
- Level-Wahl per Config (Tutorial/Second/Boss):
```java
switch (config.getString("currentLevel")) {
    case "Tutorial" -> setCurrentLevel(new TutorialLevel());
    case "Second"   -> setCurrentLevel(new SecondLevel());
    case "Boss"     -> setCurrentLevel(new BossLevel());
}
```

## 2. Setup &amp; Start
- Voraussetzungen: JDK 19 (Corretto), JavaFX 17, Maven; Assets liegen unter `/assets`.
- Build (alle Module):
```bash
mvn clean install
```
- Start (empfohlen aus der IDE):
  - Server: `de.cyzetlc.hsbi.Server` (&ouml;ffnet TCP 25570, nutzt `server/config.json`).
  - Client: `de.cyzetlc.hsbi.game.Game` (l&auml;dt Config, verbindet zum Server, zeigt Loading -> MainMenu).
- Konfiguration (`config.json`, sensible Werte als Platzhalter):
```json
{
  "currentLevel": "Tutorial",
  "soundVolume": 0.6,
  "soundMuted": false,
  "messages": { "game.prefix": "HSBI-GAME" },
  "mysql": {
    "hostname": "db-host",
    "database": "hsbi",
    "username": "user",
    "password": "<secret>",
    "port": 3306,
    "poolSize": 3
  },
  "tutorialFinished": false
}
```

## 3. Screens &amp; Navigation
- Flow: `Loading` -> `MainMenu` -> {`Settings` | `Multiplayer` | `Spiel starten`} -> `GameScreen` -> `LevelFinished` -> {n&auml;chstes Level | Hauptmen&uuml;}.
- **LoadingScreen**: animierter Background + Fortschrittsbalken; l&auml;dt alle `Material`-Texturen im Hintergrund und springt dann ins Men&uuml;.
- **MainMenuScreen**: Parallax-Background, Buttons (Start/Mehrspieler/Einstellungen/Beenden), Achievements-Panel (Dummy-Progress).
- **MultiplayerScreen**: Serverliste-Mock (Slots, MOTD); Navigation zur&uuml;ck per Button.
- **SettingsScreen**: Panel rechts oben mit Lautst&auml;rke-Slider (0-1, Style `custom-slider`) + Mute-Toggle; Werte werden in `config.json` persistiert.
- **GameScreen**: zentrales Spielfeld mit HUD, Kamera, Pause-Overlay (Logik l&auml;uft noch), Buttons Zur&uuml;ck/Pause, Quest-Text, Dateiz&auml;hler, Health-Bar.
- **LevelFinishedScreen**: Statistik-Overlay (Zeit, Punkte, Kills, Leben verloren) + Buttons f&uuml;r n&auml;chstes Level oder Hauptmen&uuml;.

## 4. Steuerung, Kamera &amp; HUD
- Bewegung: `A/D` laufen, `SPACE` springen (nur am Boden), `E` interagieren (Gas-Schranke), `W/S` im NoClip.
- Debug/States: `F1` Tooltips, `F2` Debug-Bar (FPS, Kamera, HP, States), `F3` NoClip + GodMode.
- Kamera: Dead-Zone (`marginX=400`, `marginY=150`), smoothing `cameraSmooth=0.1`, Offset f&uuml;r alle Sprites.
- HUD: Quest-Text, Files-Progress (`Files: gesammelt/gesamt`), Flipper-Hinweis, Health-Bar (Herzen basierend auf `player.getMaxHealth()`=3), Debug-Bar optional.
- Eingabe-Entprellung via `InputManager.pollJustPressed()`, Beispiel:
```java
if (screenManager.getInputManager().pollJustPressed(KeyCode.F2)) {
    showDebugBar = !showDebugBar;
}
```

## 5. Gameplay-Objekte &amp; Mechaniken
- **Player &amp; States**: `EntityPlayer` erbt `Player` (Basis-HP 3, NameTag, Richtung f&uuml;r Sprite-Flip, Flag `canCollectFiles`); GodMode/NoClip blocken Schaden &amp; Kollision.
- **Items &amp; Sammelobjekte**: USB-Stick schaltet Files-Sammeln frei; `FolderBlock` z&auml;hlt nur nach Unlock; `FlipperItem` setzt Schl&uuml;ssel-Flag (HUD-Hinweis) f&uuml;r Gas-Schranke; `ServerBlock` animiertes Deko-Objekt.
- **Perks**: `JumpBoostBlock` und `SpeedBoostBlock` erh&ouml;hen Sprungkraft/MoveSpeed um 25% f&uuml;r 10s (via `PauseTransition`), deaktivieren sich danach.
- **Gefahren &amp; Gegner**: `LavaBlock` zieht 0.25 HP pro Tick (kein Schaden in GodMode); `GasBarrierBlock` t&ouml;tet/blockiert bis E + Flipper, blendet per Fade aus; `RobotEnemyBlock` patrouilliert, schie&szlig;t `LaserBlock` (4s Lebenszeit, 1 DMG, ignoriert GodMode beim Schaden); BossLevel nutzt mehrere Roboter.
- **Plattformen**: `Platform` (statisch, Tileset), `FloatingPlatformBlock` (pendelt Start/Ende, Animation, tr&auml;gt Spieler &uuml;ber `deltaX`), `EntityMovingPlatform` (Sinus-Hover + Ping-Pong, unterst&uuml;tzt Landen).

## 6. Leveldesign
- **TutorialLevel**: gestaffelte Plattformen, Lava-L&uuml;cken gef&uuml;llt, USB-Stick fr&uuml;h, Jump/Speed-Boost, Server-Block, FloatingPlatform zwischen Inseln, Finish rechts; `onFinish()` setzt `tutorialFinished=true`.
- **SecondLevel**: dichtere Plattformen, 8 Folder, JumpBoost-Route, FlipperItem nahe Start, Boss-Roboter auf Endplattform, Finish am rechten Rand; Gas-Schranke vorbereitet (aktuell auskommentiert), einmaliger Spawn-Sound.
- **BossLevel**: breite Startplattform, vier RobotEnemyBlocks als Mini-Boss-Gasse, Finish dahinter.
- Aktuelles Startlevel &uuml;ber `config.json` (`currentLevel`), Next-Level-Kette: Tutorial -> Second -> Boss -> Ende.

## 7. Audio &amp; Settings
- `SoundManager`: Caching f&uuml;r Media, globale Lautst&auml;rke (`setVolume` 0-1), Mute-Flag, Background-Musik (`playBackground(Music.MENU, true)`), Ducking f&uuml;r Effekte (`playWithDuck`).
- SettingsScreen spiegelt Sound-State: Slider aktualisiert Volume live, Mute deaktiviert Slider und toggelt Text (`Sound an`/`Stummschalten`), persistiert direkt in der Config.
- Effektsounds: Klicks bei Sammeln/Interaktion, Buff-Sounds, Laser/Robot; Musik l&auml;uft men&uuml;weit im Loop.
- Asset-Laden: LoadingScreen w&auml;rmt alle Block-Texturen vor, um Ruckler im ersten Frame zu vermeiden.

## 8. Netzwerk &amp; Daten
- Client baut TCP-Verbindung zu `localhost:25570`, sendet Login (`ClientLoginPacket`) und Herzschlag (`ClientDataPacket` alle 50ms).
- Events: `ReceivePacketEvent` + Listener (Packet/UserMessage/Player) verteilen eingehende Pakete im Client.
- Score-&Uuml;bertragung: `sendFinalScore(long)` verschickt `ClientSubmitScorePacket`; Server kann via `ServerSendHighscoresPacket` Leaderboards broadcasten (Hook vorhanden).
- Server: MultiClientHandler pro Socket, sendet periodisch Statusnachricht, deserialisiert Pakete via `SerializationUtils`, nutzt `JsonConfig` + `QueryHandler` (MySQL-Creds in Config) f&uuml;r DB-Anbindung.
- Packets sind serialisiert (max ~0.5 MB) und werden &uuml;ber das Event-System abbrechbar (`EventCancelable`).

## 9. Debugging &amp; To-dos
- Live-Checks: F1/F2/F3-Toggles, Pause-Overlay (Hinweis: Logik pausiert noch nicht), Health-Bar reagiert auf Schaden/Heilung, Files-Z&auml;hler nach USB-Stick.
- Manuelle Tests:
  - Tutorial: USB aufnehmen -> Folder einsammeln -> Finish l&ouml;st LevelFinishedScreen aus.
  - Buffs: Jump/Speed l&ouml;sen 10s-Reset aus, Werte fallen korrekt zur&uuml;ck.
  - Gegner: RobotEnemyBlock l&auml;sst sich stompen (von oben) oder verursacht Schaden, Laser verschwinden nach 4s.
  - Flipper + GasBarrier (falls platziert) deaktiviert Schranke per E und blendet aus.
- Offene Punkte: echte Pause (Update-Schleife stoppen), `Level.update()` nutzen f&uuml;r bewegte Bl&ouml;cke, Kamera-Clamps pro Level, Statistikwerte auf echte Daten mappen, Multiplayer-/Highscore-Anbindung an Server fertigstellen.